In [16]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from imblearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import classification_report, ConfusionMatrixDisplay, recall_score
from sklearn.decomposition import PCA
from scipy.stats import ttest_ind
from imblearn.over_sampling import SMOTE
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer


In [17]:
df = pd.read_csv("dados\dados_tratados.csv")

In [18]:
df.columns.tolist()

['idade',
 'renda_anual',
 'status_moradia_atual',
 'tempo_emprego',
 'proposito_emprestimo',
 'risco_cliente',
 'valor_total_emprestimo',
 'taxa_juros',
 'proporcao_emprestimo_renda',
 'negativado',
 'tempo_relacionamento_bancario',
 'status_emprestimo',
 'razao_tempo_emprego_banco',
 'faixa_etaria']

In [19]:
def clf_metrics_com_return(modelo, X, y_true, label, plot_conf_mat=True, print_cr=True):
    
    if print_cr:
        print(f"\nMétricas de avaliação de {label}:\n")
    
    y_pred = modelo.predict(X)

    if plot_conf_mat:
        fig, ax = plt.subplots(1, 2, figsize=(12, 4))

        ConfusionMatrixDisplay.from_predictions(y_true, y_pred, ax=ax[0]) 
        ConfusionMatrixDisplay.from_predictions(y_true, y_pred, normalize="all", ax=ax[1])
        plt.show()

    if print_cr:
        print(classification_report(y_true, y_pred))
    
    return classification_report(y_true, y_pred, output_dict=True)

In [20]:
class OutlierDetector(BaseEstimator, TransformerMixin):
    def __init__(self, col_categoria, col_valor, faixa_de_tolerancia=1.5):
        self.col_categoria = col_categoria
        self.col_valor = col_valor
        self.faixa_de_tolerancia = faixa_de_tolerancia
        self.limites_outliers = {}

    def fit(self, X, y=None):
        # Calcula os limites de outliers (IQR) para cada categoria do propósito
        for categoria in X[self.col_categoria].unique():
            subset = X[X[self.col_categoria] == categoria][self.col_valor]
            Q1 = subset.quantile(0.25)
            Q3 = subset.quantile(0.75)
            IQR = Q3 - Q1
            limite_inferior = Q1 - self.faixa_de_tolerancia * IQR
            limite_superior = Q3 + self.faixa_de_tolerancia * IQR
            self.limites_outliers[categoria] = (limite_inferior, limite_superior)
        return self

    def transform(self, X, y=None):
        # Cria uma coluna para marcar os outliers como 1 ou 0
        X = X.copy()
        X['outlier_valor_proposito'] = X.apply(
            lambda row: 1 if (row[self.col_valor] < self.limites_outliers[row[self.col_categoria]][0] or
                              row[self.col_valor] > self.limites_outliers[row[self.col_categoria]][1])
            else 0, axis=1
        )
        return X

In [21]:
colunas_categoricas = ['proposito_emprestimo', 'status_moradia_atual', 'faixa_etaria']
colunas_numericas = df.drop(columns=['status_emprestimo', "negativado"]).select_dtypes(include='number').columns.tolist()
# Tirei negativado pq ele também é binário (0, 1)


# Configurando o OneHotEncoder no ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(drop=None, sparse_output=False, handle_unknown='ignore'), colunas_categoricas, ),
        ('num', StandardScaler(), colunas_numericas)
    ],
    remainder='passthrough'  # Mantém as outras colunas sem mudanças
)

In [22]:
X = df.drop(columns=['status_emprestimo'], axis=1)
y = df['status_emprestimo']

X_teste, X_treino, y_teste, y_treino = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y, shuffle=True)

In [23]:
# Vou fazer alguns testes aqui. Tanto implementar balanceamento dentro dos hiperparâmetros dos estimadores, como SMOTE



# Regressão logística
pipe_logit_cru = Pipeline([('outlier_detector', OutlierDetector(col_categoria='proposito_emprestimo', col_valor='valor_total_emprestimo')),
                        ('preprocessor', preprocessor),
                       ("logit", LogisticRegression(random_state=42))])

pipe_logit_balanceado = Pipeline([('outlier_detector', OutlierDetector(col_categoria='proposito_emprestimo', col_valor='valor_total_emprestimo')),
                                  ('preprocessor', preprocessor),
                       ("logit", LogisticRegression(random_state=42, class_weight='balanced'))])

pipe_logit_smote = Pipeline([('outlier_detector', OutlierDetector(col_categoria='proposito_emprestimo', col_valor='valor_total_emprestimo')),
                             ('preprocessor', preprocessor),
                             ("smote", SMOTE(random_state=42)),
                       ("logit", LogisticRegression(random_state=42))])


# Random Forest
pipe_rf_cru = Pipeline([('outlier_detector', OutlierDetector(col_categoria='proposito_emprestimo', col_valor='valor_total_emprestimo')),
                        ('preprocessor', preprocessor),
                    ("rf", RandomForestClassifier(random_state=42))])

pipe_rf_balanceado = Pipeline([('outlier_detector', OutlierDetector(col_categoria='proposito_emprestimo', col_valor='valor_total_emprestimo')),
                               ('preprocessor', preprocessor),
                    ("rf", RandomForestClassifier(random_state=42, class_weight='balanced'))])

pipe_rf_smote = Pipeline([('outlier_detector', OutlierDetector(col_categoria='proposito_emprestimo', col_valor='valor_total_emprestimo')),
                          ('preprocessor', preprocessor),
                          ("smote", SMOTE(random_state=42)),
                    ("rf", RandomForestClassifier(random_state=42))])

# Decision Tree
pipe_dt_cru = Pipeline([('outlier_detector', OutlierDetector(col_categoria='proposito_emprestimo', col_valor='valor_total_emprestimo')),
                        ('preprocessor', preprocessor),
                    ("dt", DecisionTreeClassifier(random_state=42))])

pipe_dt_balanceado = Pipeline([('outlier_detector', OutlierDetector(col_categoria='proposito_emprestimo', col_valor='valor_total_emprestimo')),
                               ('preprocessor', preprocessor),
                    ("dt", DecisionTreeClassifier(random_state=42, class_weight='balanced'))])

pipe_dt_smote = Pipeline([('outlier_detector', OutlierDetector(col_categoria='proposito_emprestimo', col_valor='valor_total_emprestimo')),
                              ('preprocessor', preprocessor),
                          ("smote", SMOTE(random_state=42)),
                    ("dt", DecisionTreeClassifier(random_state=42))])



# SVM

pipe_svm_cru = Pipeline([('outlier_detector', OutlierDetector(col_categoria='proposito_emprestimo', col_valor='valor_total_emprestimo')),
                         ('preprocessor', preprocessor),
                     ("svm", SVC(random_state=42))])

pipe_svm_balanceado = Pipeline([('outlier_detector', OutlierDetector(col_categoria='proposito_emprestimo', col_valor='valor_total_emprestimo')),
                                ('preprocessor', preprocessor),
                     ("svm", SVC(random_state=42, class_weight='balanced'))])

pipe_svm_smote = Pipeline([('outlier_detector', OutlierDetector(col_categoria='proposito_emprestimo', col_valor='valor_total_emprestimo')),
                           ('preprocessor', preprocessor),
                           ("smote", SMOTE(random_state=42)),
                     ("svm", SVC(random_state=42))])


# Catboost 

pipe_catboost_cru = Pipeline([('outlier_detector', OutlierDetector(col_categoria='proposito_emprestimo', col_valor='valor_total_emprestimo')),
                              ('preprocessor', preprocessor),
                              ("catboost", CatBoostClassifier(random_state=42, verbose=0))])

pipe_catboost_balanceado = Pipeline([('outlier_detector', OutlierDetector(col_categoria='proposito_emprestimo', col_valor='valor_total_emprestimo')),
                                     ('preprocessor', preprocessor),
                                     ("catboost", CatBoostClassifier(random_state=42, auto_class_weights='Balanced', verbose=0))])

# CatBoost com SMOTE
pipe_catboost_smote = Pipeline([('outlier_detector', OutlierDetector(col_categoria='proposito_emprestimo', col_valor='valor_total_emprestimo')),
                                ('preprocessor', preprocessor),
                                   ("smote", SMOTE(random_state=42)),
                                   ("catboost", CatBoostClassifier(random_state=42, verbose=0))])

# ================================================


dict_pipes = {"logit cru" : pipe_logit_cru,
              "logit balanceado": pipe_logit_balanceado,
              "logit smote": pipe_logit_smote,
              "random forest cru" : pipe_rf_cru,
              "random forest balanceado": pipe_rf_balanceado,
              "random forest smote": pipe_rf_smote,
              "decision tree cru" : pipe_dt_cru,
              "decision tree balanceado": pipe_dt_balanceado,
              "decision tree smote": pipe_dt_smote,
              "svm cru" : pipe_svm_cru,
              "svm balanceado": pipe_svm_balanceado,
              "svm smote": pipe_svm_smote,
              "catboost_cru": pipe_catboost_cru,
              "catboost balancado": pipe_catboost_balanceado,
              "catboost smote": pipe_catboost_smote
              }

# ================================================
# experimento!
# este dicionário que vai guardar os resultados do experimento

# resultado_experimentos = {"estimador" : [],
#                           "recall_treino" : [],
#                           "recall_teste" : []}

# for label, pipe in dict_pipes.items():
    
#     pipe.fit(X_treino, y_treino)
    
#     dict_metricas_treino = clf_metrics_com_return(pipe, X_treino, y_treino, "treino", plot_conf_mat=False, print_cr=False)

#     # print(dict_metricas_treino)
    
#     dict_metricas_teste = clf_metrics_com_return(pipe, X_teste, y_teste, "teste", plot_conf_mat=False, print_cr=False)
    
#     f1_treino = dict_metricas_treino["1"]["recall"]
#     f1_teste = dict_metricas_teste["1"]["recall"]
    
#     resultado_experimentos["estimador"].append(label)
#     resultado_experimentos["recall_treino"].append(f1_treino)
#     resultado_experimentos["recall_teste"].append(f1_teste)

#     print(f'treinamento do modelo {label} finalizado')
    
# df_results = pd.DataFrame(resultado_experimentos)

# # em casos de underfit
# df_results["gap"] = (df_results["recall_treino"] - df_results["recall_teste"]).apply(lambda x: x if x > 0 else np.inf)

# df_results = df_results.sort_values("recall_teste", ascending=False)

# df_results

In [ ]:
# Mesma coisa usando CV

# Configuração do StratifiedKFold
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Inicialização do dicionário de resultados
resultado_experimentos = {"estimador": [], "recall_treino": [], "recall_teste": []}

# Loop pelos pipelines
for nome_modelo, pipeline in dict_pipes.items():
    recall_treino_lista = []
    recall_teste_lista = []
    
    # Realiza a validação cruzada estratificada
    for indice_treino, indice_valida in kf.split(X_treino, y_treino):
        X_treino_split, X_valida_split = X_treino.iloc[indice_treino], X_treino.iloc[indice_valida]
        y_treino_split, y_valida_split = y_treino.iloc[indice_treino], y_treino.iloc[indice_valida]
        
        # Treina o modelo
        pipeline.fit(X_treino_split, y_treino_split)
        
        # Avalia o modelo no treino
        y_pred_treino = pipeline.predict(X_treino_split)
        recall_treino = recall_score(y_treino_split, y_pred_treino, average='binary')
        recall_treino_lista.append(recall_treino)
        
        # Avalia o modelo no teste (validação)
        y_pred_valida = pipeline.predict(X_valida_split)
        recall_teste = recall_score(y_valida_split, y_pred_valida, average='binary')
        recall_teste_lista.append(recall_teste)
    
    # Calcula a média dos recalls
    recall_treino_medio = np.mean(recall_treino_lista)
    recall_teste_medio = np.mean(recall_teste_lista)
    
    # Armazena os resultados
    resultado_experimentos["estimador"].append(nome_modelo)
    resultado_experimentos["recall_treino"].append(recall_treino_medio)
    resultado_experimentos["recall_teste"].append(recall_teste_medio)
    
    print(f'Treinamento do modelo {nome_modelo} finalizado')

# Resultado final em DataFrame
df_resultados = pd.DataFrame(resultado_experimentos)

# Em casos de underfit, calcula a diferença entre o recall de treino e teste
df_resultados["gap"] = (df_resultados["recall_treino"] - df_resultados["recall_teste"]).apply(lambda x: x if x > 0 else np.inf)

# Ordena os resultados pelo recall no teste
df_resultados = df_resultados.sort_values("recall_teste", ascending=False)

df_resultados

Treinamento do modelo logit cru finalizado
Treinamento do modelo logit balanceado finalizado
Treinamento do modelo logit smote finalizado
Treinamento do modelo random forest cru finalizado
Treinamento do modelo random forest balanceado finalizado
Treinamento do modelo random forest smote finalizado
Treinamento do modelo decision tree cru finalizado
Treinamento do modelo decision tree balanceado finalizado
Treinamento do modelo decision tree smote finalizado


In [ ]:
# transformar idade em uma coluna faixa etária e dividir em bins
# Deixar para transformar no one hot enconder só depois pq enquanto ainda for faixa vai ser utilizado
# para pegar outlier de renda ou valor total do empréstimo